In [1]:
#conda list
import pandas as pd
import numpy as np
import time

Prepare a log.txt if not exists

https://www.freecodecamp.org/news/file-handling-in-python/#:~:text=In%20Python%2C%20you%20use%20the,it%20will%20return%20an%20error.

In [2]:
#
f = open("log.txt","a")

In [3]:
import os.path
#https://stackoverflow.com/questions/82831/how-do-i-check-whether-a-file-exists-without-exceptions
if os.path.isfile('q_table_maze.csv'):
    print('Q-Table CSV Exists.')
    time.sleep(1)
    mode=input('1 for loading trained Q-table,2 for starting afresh. Please choose:')
    
    if mode=="1":
        #当后续的训练想要使用之前得到的Q-table时，导出后，传参给RL_brain.py中的__init__ function，最后赋值给self.q_table
        df=pd.read_csv('q_table_maze.csv',index_col=0)
        #df=pd.read_csv('q_table_output.csv') 默认读取时会自动设置新的index，但是这里我们需要的是将第一列(state)设置为index
        #df.columns发现输出时columns的dtype是int，而现在导入变成了object类型
        df.columns=[0,1,2,3]
    elif mode=="2":
        df=pd.DataFrame(columns=[0,1,2,3],dtype=np.float64) #initialize table
        f.write("\n\n-----Start Afresh-----\n")
else:
    df=pd.DataFrame(columns=[0,1,2,3],dtype=np.float64) #initialize table
    f.write("\n\n-----Start Afresh-----\n")

Q-Table CSV Exists.
1 for loading trained Q-table,2 for starting afresh. Please choose:1


In [4]:
# have a look at current Q-table
df

,0,1,2,3
"[5.0, 5.0, 35.0, 35.0]",0.000000e+00,4.675968e-10,1.643093e-15,0.000000e+00
"[45.0, 5.0, 75.0, 35.0]",0.000000e+00,2.272535e-13,-8.910000e-05,0.000000e+00
"[5.0, 45.0, 35.0, 75.0]",0.000000e+00,5.263874e-08,-9.000000e-05,7.702119e-11
"[5.0, 85.0, 35.0, 115.0]",2.802498e-10,3.554197e-06,-1.000000e-02,1.217314e-09
terminal,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
"[5.0, 125.0, 35.0, 155.0]",0.000000e+00,2.865189e-08,1.689263e-04,0.000000e+00
"[45.0, 125.0, 75.0, 155.0]",0.000000e+00,4.795443e-06,5.548862e-03,0.000000e+00
"[45.0, 45.0, 75.0, 75.0]",-8.100000e-07,-1.000000e-02,-1.990000e-02,1.654124e-11
"[85.0, 5.0, 115.0, 35.0]",0.000000e+00,-1.990000e-02,0.000000e+00,0.000000e+00
"[125.0, 5.0, 155.0, 35.0]",0.000000e+00,-9.000000e-05,0.000000e+00,0.000000e+00


In [5]:
from maze_env import Maze
from RL_brain import SarsaTable
import time

In [6]:
epoches=10
def update():
    count_success=0
    count_steps=0
    start=time.time()
    for episode in range(epoches):
        # 初始化环境
        observation = env.reset()

        # Sarsa 根据 state 观测选择行为
        action = RL.choose_action(str(observation))
        
        #step counter
        steps=0

        while True:
            # 刷新环境
            env.render()

            # 在环境中采取行为, 获得下一个 state_ (obervation_), reward, 和是否终止
            observation_, reward, done = env.step(action)

            # 根据下一个 state (obervation_) 选取下一个 action_
            action_ = RL.choose_action(str(observation_))

            # 从 (s, a, r, s, a) 中学习, 更新 Q_tabel 的参数 ==> Sarsa
            RL.learn(str(observation), action, reward, str(observation_), action_)

            # 将下一个当成下一步的 state (observation) and action
            observation = observation_
            action = action_
            
            steps+=1

            # 终止时跳出循环
            if done:
                end=time.time()
                if reward==1:
                    count_success+=1
                count_steps+=steps
                print('Episode '+str(episode)+', total steps: '+str(steps)+', reward: ', end='')
                print('1') if reward==1 else print ('-1')
                break

    # 大循环完毕
    print('game over, AvgSteps= '+str(round(count_steps/epoches))+', success rate= '+str("%.2f%%"%(count_success/epoches*100)))
    # write into log.txt (append)
    f.writelines([time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime(start)),"UTC——",
                  time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime(end)),"UTC\n","Epoches: ",str(epoches)
                  ,", AvgSteps: ",str(round(count_steps/epoches)),", success rate: ",str("%.2f%%"%(count_success/epoches*100)),"\n"])
    f.close()
    env.destroy()

if __name__ == "__main__":
    env = Maze()
    #RL = SarsaTable(actions=list(range(env.n_actions)))
    RL = SarsaTable(actions=list(range(env.n_actions)),trained_q_table=df)

    env.after(epoches, update)
    env.mainloop()
    print('\r\nQ-table:\n')
    print(RL.q_table)


Episode 0, total steps: 6, reward: 1
Episode 1, total steps: 7, reward: 1
Episode 2, total steps: 6, reward: 1
Episode 3, total steps: 6, reward: 1
Episode 4, total steps: 7, reward: 1
Episode 5, total steps: 8, reward: 1
Episode 6, total steps: 8, reward: 1
Episode 7, total steps: 10, reward: 1
Episode 8, total steps: 78, reward: 1
Episode 9, total steps: 29, reward: 1
game over, AvgSteps= 16, success rate= 100.00%

Q-table:

                                         0             1             2  \
[5.0, 5.0, 35.0, 35.0]       -7.217097e-09 -8.038881e-07 -7.938787e-07   
[45.0, 5.0, 75.0, 35.0]       0.000000e+00 -6.719915e-09 -8.732691e-05   
[5.0, 45.0, 35.0, 75.0]       0.000000e+00  1.447471e-06 -8.821609e-05   
[5.0, 85.0, 35.0, 115.0]      6.413264e-09  4.266566e-05 -1.000000e-02   
terminal                      0.000000e+00  0.000000e+00  0.000000e+00   
[5.0, 125.0, 35.0, 155.0]     0.000000e+00  2.865189e-08  9.992260e-04   
[45.0, 125.0, 75.0, 155.0]    0.000000e+00  9.32328

Note：浮点数精度丢失问题
https://www.zhihu.com/question/28551135

In [7]:
df=RL.q_table

In [8]:
df

,0,1,2,3
"[5.0, 5.0, 35.0, 35.0]",-7.217097e-09,-8.038881e-07,-7.938787e-07,0.000000e+00
"[45.0, 5.0, 75.0, 35.0]",0.000000e+00,-6.719915e-09,-8.732691e-05,0.000000e+00
"[5.0, 45.0, 35.0, 75.0]",0.000000e+00,1.447471e-06,-8.821609e-05,8.331520e-10
"[5.0, 85.0, 35.0, 115.0]",6.413264e-09,4.266566e-05,-1.000000e-02,1.217314e-09
terminal,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
"[5.0, 125.0, 35.0, 155.0]",0.000000e+00,2.865189e-08,9.992260e-04,0.000000e+00
"[45.0, 125.0, 75.0, 155.0]",0.000000e+00,9.323283e-05,1.819892e-02,0.000000e+00
"[45.0, 45.0, 75.0, 75.0]",-8.100000e-07,-1.000000e-02,-1.990000e-02,-7.539836e-07
"[85.0, 5.0, 115.0, 35.0]",0.000000e+00,-1.990000e-02,0.000000e+00,-7.859413e-07
"[125.0, 5.0, 155.0, 35.0]",0.000000e+00,-8.910000e-05,-8.100000e-07,-7.144929e-09


In [9]:
# saving the dataframe
df.to_csv('q_table_maze.csv')